In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gtsa

### Plot a single COG

In [3]:
cog_url = 'http://conus-historicals.s3.amazonaws.com/rainier/orthos/ortho_1955-09-15_EE_reproj_COG.tif'

In [4]:
m = gtsa.plotting.plot_cog(cog_url   = cog_url,
                       zoom_start = 10)
m

In [5]:
# m.save('mount_rainier_1955-09-15.html')

### Plot multiple cogs

In [6]:
s3_bucket_name = 'conus-historicals'
folder_path = 'rainier/orthos'

In [7]:
cog_urls = gtsa.io.parse_urls_from_S3_bucket(s3_bucket_name,
                                             folder = folder_path,
                                             aws_server_url = 's3.amazonaws.com')

dates = gtsa.io.parse_timestamps(cog_urls,
                                 date_string_pattern='....-..-..')

In [8]:
map_center_lon = -121.7603
map_center_lat = 46.8523

m = gtsa.plotting.plot_cogs(cog_urls,
                        dates,
                        map_center_lon      = map_center_lon,
                        map_center_lat      = map_center_lat,
                        zoom_start          = 11,
                        overview_cog_index  = 1,
                        print_info          = True,
                       )
m

map center: -121.7603 46.8523


In [9]:
# m.save('mount_rainier_all.html')

### Plot multiple sites

In [10]:
s3_bucket_name = 'conus-historicals'

In [11]:
sites = ['rainier','baker','scg']
folders = [gtsa.io.Path(s, 'orthos').as_posix() for s in sites]

site_marker_names = ['Mount Rainier','Mount Baker', 'South Cascade']
site_marker_coords = [(46.8523, -121.7603),(48.7767, -121.8144), (48.3717, -121.0660)]
cog_overview_indices = [1,0,0]


map_center = (47.6062, -122.3321)

In [12]:
payload = {}
payload['map_center'] = map_center
payload['sites'] = {}

for i, site in enumerate(sites):
    payload['sites'][site] = {'marker_name':site_marker_names[i],
                              'marker_coords': site_marker_coords[i],
                     'overview_index': cog_overview_indices[i],
                     'cog_urls': gtsa.io.parse_urls_from_S3_bucket(s3_bucket_name,
                                                                   folder = folders[i],
                                                                   aws_server_url = 's3.amazonaws.com'),
                             }
    payload['sites'][site]['dates'] = gtsa.io.parse_timestamps(payload['sites'][site]['cog_urls'])
    payload['sites'][site]['cog_names'] = [site +'_'+ x for x in payload['sites'][site]['dates']]

In [13]:
m = gtsa.plotting.plot_cogs_sites(payload,
                                  zoom_start           = 7,
                                  print_info          = True,
                             )
m

map center: -122.3321 47.6062


In [14]:
# m.save('conus_sites.html')

Example https://staff.washington.edu/knuth/downloads/conus_sites.html

### Known issues
- collapsable navigation menu by site would be preferred but is not natively supported by folium
- duplicate dates will overwrite. stopgap solution is to prefix with the site id
- the All Sites legend entry is a stopgap solution to only show a single COG to start
- if COGs are not added to a featuregroup that has show = False, they get burned in to the map somehow, which appears to be a bug